In [1]:
import warnings

warnings.filterwarnings("ignore")
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
import time
import os
import json
import numpy as np
import random


%load_ext nb_black

# selenium _ beautusoup _ request

<IPython.core.display.Javascript object>

In [2]:
def driver_operation(url, size):
    # options = webdriver.ChromeOptions()  # I would also suggest to use this instead of Options()
    # options.add_argument("--headless")
    # options.add_argument("--enable-javascript")  # To be on the safe side, although it seems to be enabled by default

    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    #options.headless = True

    driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver.exe", options=options)
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    driver.get(url)
    time.sleep(1)

    for i in range(0, size+1):
        driver.execute_script("window.scrollTo({start}*document.body.scrollHeight/{r}, {end}*document.body.scrollHeight/{r}-100)".format(start=i, end = i+1, r=size))
        time.sleep(1.5)

    return driver







#scroll_to_bottom(driver)

<IPython.core.display.Javascript object>

In [12]:
def scrape_restaurants(base_url, location, driver):

    cat = []
    try: 
        temp_urls = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/main/div[2]/div[3]').find_elements_by_tag_name("a")
        for url in temp_urls:
            cat.append(url.get_attribute('href'))
    except:
        print("Skipped " + cat)


    with open('temp_urls.txt', 'a') as f:
        cat_str = list(map(lambda x: str(x)+'\n', cat))
        for i in cat_str:
            f.write(i)


    lines_seen = set()  # holds lines already seen
    out_file = open(location + "_restaurant_urls.txt", "w+")
    for line in open("temp_urls.txt", "r"):
        if line not in lines_seen:  # not a duplicate
            out_file.write(line)
            lines_seen.add(line)
    out_file.close()
    os.remove("temp_urls.txt")

    with open(location + "_restaurant_urls.txt") as file:
        categories_lst = file.readlines()

    categories_lst = [str(i)[:-1] for i in categories_lst]

    # driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver.exe")
    # driver.maximize_window()
    # driver.set_window_size(1920, 1080)

    restaurant_arr = []
    for url in categories_lst:
        driver.get(url)
        restaurant_lst = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[5]/div").find_elements_by_tag_name("a")
        for i in restaurant_lst:
            restaurant_arr.append(i.get_attribute('href'))



    with open('restaurant_new_test.txt', 'a') as f:
        restaurant_str2 = list(map(lambda x: str(x)+'\n', restaurant_arr))
        for i in restaurant_str2:
            f.write(i)

    os.remove('temp_urls.txt')



<IPython.core.display.Javascript object>

In [4]:
# base_url = "https://www.ubereats.com/ca/category/"
# city_list = ["toronto-on", "hamilton-on", "kingston-on"]

# driver = driver_operation(base_url, 14)
# for city in city_list:
#     scrape_restaurants(base_url, city, driver)

<IPython.core.display.Javascript object>

In [5]:
# base_url = "ubereats.com/ca/category/toronto-on"
# city_list = ["toronto-on", "hamilton-on", "kingston-on"]

# #driver = driver_operation(base_url, 16)

# scrape_restaurants(base_url, 'toronto-on')

<IPython.core.display.Javascript object>

### MAIN

In [6]:

def list_urls(file):
    with open(file, 'r') as f:
        lst_restaurants = f.readlines()
    lst_restaurants = [i.replace('\n', '') for i in lst_restaurants if '\n' in i]
    return lst_restaurants

<IPython.core.display.Javascript object>

In [54]:
def get_restaurant_url(base_url, driver, city):  
    cat = []

    title = ''
    image = ''
    contingency_image = ''
    restaurant_url = ''
    df_restaurant = pd.DataFrame(columns = ['title', 'img_url', 'img_url_contingency', 'restaurant_url', 'category', 'city'])


    try:
        xpath = '/html/body/div[1]/div[1]/div/main/div[5]/div'
        list_restaurants_div = driver.find_element_by_xpath(xpath).find_element_by_tag_name("div")
        class_name = list_restaurants_div.get_attribute("class").replace(' ','.')
        list_restaurants = driver.find_element_by_xpath(xpath).find_elements_by_class_name(class_name)
        len(list_restaurants)

        for i in range(1, len(list_restaurants)+1):
            # get restaurant name
            try: 
                title = str(driver.find_element_by_xpath(
                                                "/html/body/div[1]/div[1]/div/main/div[5]/div/div[" + str(i) + "]/div/a/h3").text)
            except:
                titlee = ''


            # get avt restaurant
            try: 
                image = str(driver.find_element_by_xpath(
                                                "/html/body/div[1]/div[1]/div/main/div[5]/div/div[" + str(i) + "]/div[1]/div/div/figure/div/div/picture/source").get_attribute("srcset"))
                                                
            except: 
                try:  
                    image = str(driver.find_element_by_xpath(
                                                "/html/body/div[1]/div[1]/div/main/div[5]/div/div[" + str(i) + "]/div[1]/div/div/figure/div/div/picture/img").get_attribute("src"))
                
                except:
                    image = ''

            # get contingency avt restaurant
            try: 
                contingency_image = str(driver.find_element_by_xpath(
                                                "/html/body/div[1]/div[1]/div/main/div[5]/div/div[" + str(i) + "]/div[1]/div/div/figure/div/div/picture/img").get_attribute("src"))
            except:
                contingency_image = ''
                
            # get restaurant url
            try:  
                restaurant_url = str(driver.find_element_by_xpath(
                                            "/html/body/div[1]/div[1]/div/main/div[5]/div/div[" + str(i) + "]/div/a").get_attribute('href'))
            except:
                restaurant_url = ''

            df_restaurant = df_restaurant.append({
            'title': title,
            'img_url': image,
            'img_url_contingency': contingency_image,
            'restaurant_url': restaurant_url,
            'category': base_url.split('/')[-1],
            'city': city,
            }, ignore_index=True)
    except:
        print('skip'+base_url)
        with open('failed_restaurant_toronto.txt', 'a') as f:
            f.write(base_url + '\n')
        driver.close()
    driver.close()
    return df_restaurant



<IPython.core.display.Javascript object>

In [55]:
# lst_urls_test = list_urls('hamilton-on_restaurant_urls.txt')
# len(lst_urls_test)

<IPython.core.display.Javascript object>

In [56]:
# df = pd.DataFrame(columns = ['title', 'img_url', 'restaurant_url', 'category', 'city'])
# df.to_csv('restaurant_hamiton.csv', encoding='utf-8-sig', mode='w', index=False)

# for url in lst_urls_test:
#     driver = driver_operation(url, 14)
#     df = get_restaurant_url(url, driver, 'hamilton')
#     if df is not None:
#         df.to_csv('restaurant_hamiton.csv', encoding='utf-8-sig', mode='a', header=False, index=False)
#         df = df.iloc[0:0]

<IPython.core.display.Javascript object>

### Toronto

In [57]:
#### Toronto

<IPython.core.display.Javascript object>

In [58]:
lst_urls_test = list_urls('toronto-on_restaurant_urls.txt')
len(lst_urls_test)

198

<IPython.core.display.Javascript object>

In [59]:
df = pd.DataFrame(columns = ['title', 'img_url', 'img_url_contingency', 'restaurant_url', 'category', 'city'])
df.to_csv('restaurant_toronto.csv', encoding='utf-8-sig', mode='w', index=False)

for url in lst_urls_test:
    driver = driver_operation(url, 10)
    df = get_restaurant_url(url, driver, 'toronto')
    if df is not None:
        df.to_csv('restaurant_toronto.csv', encoding='utf-8-sig', mode='a', header=False, index=False)
        df = df.iloc[0:0]
        break

<IPython.core.display.Javascript object>